Install Libraries

In [1]:
!pip install pybaseball pandas numpy xgboost scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.2/409.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 37.9 MB/s eta 0:00:00


Import Libraries

In [2]:
import pandas as pd
import numpy as np
from pybaseball import statcast
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor


Define Function to get training data

In [3]:
def get_statcast_data_for_year(year):
    start_date = f"{year}-04-01"
    end_date = f"{year}-10-01"
    print(f"Fetching data for {year} season...")
    return statcast(start_date, end_date)

# Fetch data for 2021-2023
years = [2021, 2022, 2023]
statcast_all_players = pd.concat([get_statcast_data_for_year(year) for year in years], ignore_index=True)


Fetching data for 2021 season...
This is a large query, it may take a moment to complete


/usr/local/lib/python3.11/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|          | 0/184 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_co

Fetching data for 2022 season...
This is a large query, it may take a moment to complete


/usr/local/lib/python3.11/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|          | 0/184 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_co

Fetching data for 2023 season...
This is a large query, it may take a moment to complete


/usr/local/lib/python3.11/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|          | 0/184 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_co

In [4]:
print(statcast_all_players.columns.tolist())


['pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'description', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 'iso_value', 'launch_speed_a

Clean and Prepare the Data

In [5]:
import pandas as pd

# 🛠 Step 1: Dynamically Detect Column Names
# This ensures the script works regardless of dataset variations.
column_mapping = {
    "batter": "player_id",
    "game_date": "date",
    "game_year": "year",
    "launch_speed": "exit_velocity",
    "launch_angle": "launch_angle",
    "woba_value": "woba",
    "estimated_woba_using_speedangle": "xwoba" if "estimated_woba_using_speedangle" in statcast_all_players.columns else "xwoba",
    "estimated_slg_using_speedangle": "xslg" if "estimated_slg_using_speedangle" in statcast_all_players.columns else "xslg",
    "estimated_ba_using_speedangle": "xba" if "estimated_ba_using_speedangle" in statcast_all_players.columns else "xba",
    "iso_value": "iso",
    "babip_value": "babip"
}

# Rename columns dynamically to avoid KeyErrors
statcast_all_players = statcast_all_players.rename(columns={k: v for k, v in column_mapping.items() if k in statcast_all_players.columns})

# 🛠 Step 2: Ensure "year" is an integer
statcast_all_players["year"] = statcast_all_players["year"].astype(int)

# 🛠 Step 3: Convert Numeric Columns to Avoid Aggregation Errors
numeric_columns = ["exit_velocity", "launch_angle", "xwoba", "xslg", "xba", "woba", "iso", "babip"]

# Convert all numeric columns to float (ignores errors to avoid crashes)
for col in numeric_columns:
    if col in statcast_all_players.columns:
        statcast_all_players[col] = pd.to_numeric(statcast_all_players[col], errors="coerce")

# Drop rows where critical numeric columns are still NaN
statcast_cleaned = statcast_all_players.dropna(subset=numeric_columns)

# 🛠 Step 4: Group by Player & Year to Get Seasonal Averages
statcast_seasonal = statcast_cleaned.groupby(["player_id", "player_name", "year"])[numeric_columns].mean().reset_index()

# ✅ Step 5: Verify Data with pandas `.head()`
print("First 5 rows of the cleaned dataset:")
print(statcast_seasonal.head())

# Check column data types to confirm correctness
print("\nColumn Data Types:")
print(statcast_seasonal.dtypes)

# Compute Hard-Hit Rate (approximation based on Exit Velocity > 95 mph)
statcast_seasonal["hard_hit_rate"] = (statcast_seasonal["exit_velocity"] > 95).astype(int)

# Adjust BABIP towards league average (MLB average BABIP ~ 0.300)
league_babip = 0.300
statcast_seasonal["babip_adjusted"] = (statcast_seasonal["babip"] * 0.75) + (league_babip * 0.25)


First 5 rows of the cleaned dataset:
   player_id       player_name  year  exit_velocity  launch_angle   xwoba  \
0     405395      Abreu, Bryan  2021           97.8          32.0    0.62   
1     405395  Alexander, Jason  2022          108.4           2.0   0.465   
2     405395     Allard, Kolby  2021          91.95         -21.5  0.0635   
3     405395      Allen, Logan  2022           96.8         -19.0    0.08   
4     405395    Almonte, Yency  2022           99.4         -33.0    0.07   

     xslg    xba  woba  iso  babip  
0   1.207  0.343   2.0  3.0    0.0  
1   0.531  0.517   0.9  0.0    1.0  
2  0.0945   0.09   0.0  0.0    0.0  
3    0.12   0.12   0.0  0.0    0.0  
4   0.108  0.106   0.0  0.0    0.0  

Column Data Types:
player_id          Int64
player_name       object
year               int64
exit_velocity    Float64
launch_angle     Float64
xwoba            Float64
xslg             Float64
xba              Float64
woba             Float64
iso              Float64
babip   

Create features for k rate and bb rate

In [6]:

# Define plate appearance events
pa_events = ["single", "double", "triple", "home_run", "strikeout", "walk", "sac_fly", "hit_by_pitch"]

# Filter for plate appearances
pa_data = statcast_all_players[statcast_all_players["events"].isin(pa_events)]

# Compute Walk Rate (BB%) and Strikeout Rate (K%)
bb_k_rates = pa_data.groupby(["player_id", "year"]).agg(
    bb_count=("events", lambda x: (x == "walk").sum()),
    k_count=("events", lambda x: (x == "strikeout").sum()),
    pa_count=("events", "count")
).reset_index()

# Compute BB% and K%
bb_k_rates["bb_rate"] = (bb_k_rates["bb_count"] / bb_k_rates["pa_count"]) * 100
bb_k_rates["k_rate"] = (bb_k_rates["k_count"] / bb_k_rates["pa_count"]) * 100

# Ensure "year" is an integer to match statcast_seasonal
bb_k_rates["year"] = bb_k_rates["year"].astype(int)

# Merge BB% and K% back into seasonal data, filling missing values with 0
statcast_seasonal = statcast_seasonal.merge(
    bb_k_rates[["player_id", "year", "bb_rate", "k_rate"]],
    on=["player_id", "year"],
    how="left"
).fillna({"bb_rate": 0, "k_rate": 0})


Train the model (XGBoost)

In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor

# Define final feature set including new features
features = ["exit_velocity", "launch_angle", "xwoba", "xslg", "xba", "bb_rate", "k_rate", "hard_hit_rate", "babip_adjusted"]
X = statcast_seasonal[[col for col in features if col in statcast_seasonal.columns]]
y = statcast_seasonal["woba"]

# Handle any remaining NaN values
X = X.fillna(0)
y = y.fillna(0)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define hyperparameter grid for tuning
param_grid = {
    "n_estimators": [200, 300, 400],
    "learning_rate": [0.03, 0.05, 0.1],
    "max_depth": [3, 4, 5],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9]
}

# Initialize XGBoost model
xgb = XGBRegressor(objective="reg:squarederror")

# Grid Search with Cross-Validation (5-fold)
grid_search = GridSearchCV(xgb, param_grid, scoring="r2", cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Use best model from tuning
best_xgb = grid_search.best_estimator_

# Print best hyperparameters
print(f"🚀 Best Parameters: {grid_search.best_params_}")

# Train the best model
best_xgb.fit(X_train, y_train)

# Evaluate Model Performance
train_score = best_xgb.score(X_train, y_train)
test_score = best_xgb.score(X_test, y_test)

print(f"🎯 Optimized Training Score: {train_score:.3f}")
print(f"📉 Optimized Test Score: {test_score:.3f}")


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
🚀 Best Parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.9}
🎯 Optimized Training Score: 0.783
📉 Optimized Test Score: 0.774


Predict Adley's wOBA with the trained model for 2024 season

In [9]:
import numpy as np

# Filter Adley’s 2023 data
adley_2023 = statcast_seasonal[(statcast_seasonal["player_id"] == 668939) & (statcast_seasonal["year"] == 2023)]
adley_avg = adley_2023[features].mean()

# Convert to NumPy array for prediction
future_features = np.array([adley_avg])

# Use the best-tuned model to predict 2024 wOBA
predicted_wOBA = best_xgb.predict(future_features)[0]

print(f"🎯 **Projected wOBA for Adley Rutschman (2024):** {predicted_wOBA:.3f}")


🎯 **Projected wOBA for Adley Rutschman (2024):** 0.316


Apply park adjustment factors

In [10]:
# Camden Yards Park Factors (Fangraphs)
park_factors = {
    "wOBA": 95,  # Slightly pitcher-friendly
    "SLG": 98,   # Below-average power park
}

# Apply park factor adjustments
predicted_wOBA += (park_factors["wOBA"] - 100) / 1000
predicted_xSLG = adley_avg["xslg"] + (park_factors["SLG"] - 100) / 1000

print(f"🏟️ Park-Adjusted wOBA: {predicted_wOBA:.3f}")
print(f"🏟️ Park-Adjusted xSLG: {predicted_xSLG:.3f}")


🏟️ Park-Adjusted wOBA: 0.311
🏟️ Park-Adjusted xSLG: 0.557
